In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.api import Holt
import statsmodels.tsa.api as smt
import datetime as dt
import itertools

In [35]:
df=pd.read_csv("train.csv")
print (df.shape)
df.head()

(913000, 4)


,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [36]:
df.tail()

,date,store,item,sales
912995,2017-12-27,10,50,63
912996,2017-12-28,10,50,59
912997,2017-12-29,10,50,74
912998,2017-12-30,10,50,62
912999,2017-12-31,10,50,82


In [37]:
dt=pd.to_datetime(df.date,format="%Y-%m-%d")
df.index=pd.DatetimeIndex(dt)
df=df.drop(['date'],axis=1)
df.head()

,store,item,sales
date,,,
2013-01-01,1,1,13
2013-01-02,1,1,11
2013-01-03,1,1,14
2013-01-04,1,1,13
2013-01-05,1,1,10


# Holt-Winters and SARIMA Models Utils

# Seasonality model

In [38]:
# dictionary for the futher selection of seasonality mode
additive={"op":lambda a,b:a+b,"inv":lambda a,b: a-b}
multiplicative={"op":lambda a,b:a*b,"inv":lambda a,b:a/b}

In [39]:
#choose seasonality model from the created dictionary
seasonality_model=additive
#seasonality_model=multiplicative

In [40]:
categorize_by_week_of_year=lambda df:df.index.dayofyear//7
categorize_by_day_of_week = lambda df: df.index.dayofweek

In [41]:
def compute_seasonality(series,categorization):
    """
    compute seasonal component parameters based on provided series
    
    :type series: pd.series
    :param categorization: Function used to split values into various 
    period of season
    :type categorization: pd.DataFrame-> some categorized type. e.g. int.
    """
    
    df=pd.DataFrame()
    df['values']=series
    df.index=series.index
    df['cat']=categorization(df)
    return df.groupby(by='cat')['values'].mean()
    

In [42]:
def alter_series_by_season(series,categorization, seasonality,op):
    """
    op: the specific seasonality model
    """
    df=pd.DataFrame()
    df['values']=series
    df['cat']=categorization(df)
    return op(df['values'],df['cat']).map(seasonality)

In [43]:
def add_seasonal_component(series,categorization,seasonality):
    """
    add previously computed seasonal component back into a deseasonalized series.
    
    :type series: pd.Series
    :param categorization: Function used to split values into various periods of the season.
    :type categorization: pd.DataFrame -> some categorical type, eg. int
    :param seasonality: value returned from compute_seasonality
    :returns: Series with added seasonal component.
    :rtype: pd.Series
    """
    return alter_series_by_season(series,categorization,seasonality,
                                  seasonality_model["inv"])
    

In [44]:
def remove_seasonal_component(series, categorization, seasonality):
    """
    Try removing a previously computed seasonal component.
    
    :type series: pd.Series
    :param categorization: Function used to split values into various periods of the season.
    :type categorization: pd.DataFrame -> some categorical type, eg. int
    :param seasonality: value returned from compute_seasonality
    :returns: Deseasonalized series.
    :rtype: pd.Series
    """
    return alter_series_by_season(series, categorization, seasonality, seasonality_model["inv"])

In [45]:
def train_and_forecast(data,categorization,trainer,forecaster,deseasonize,steps_to_forecast=90):
    """
    used for a dataset with an item in a single store
    this function is used to split input data, deseasonalizes train data, 
    training use the provided trainer
    finally, forecast using forecaster.
    
    :param data: dataset with the training data
    :param categorization: Function used to split values into various periods of the season.
    :type categorization: pd.DataFrame -> some categorical type, e.g. int
    :param trainer: Function used to train the model
    :type trainer: pd.DataFrame -> model
    :param forecaster: (model, steps) -> prediction
    :param steps_to_forecast:  number of steps to forecast
    :returns:  a dataframe with:
                                date
                                sales - true values
                                forecast - forecasted values
    """
    
    
    # prepare training and validation datasets
    df_train=data.iloc[:-365].copy() 
    df_validation=data.iloc[-365:].copy() # the last 365 rows for validation
    df_train.index=pd.DatetimeIndex(df_train['date'])
    df_validation.index=pd.DatetimeIndex(df_validation['date'])
    
    if deseasonize:
        # true or Fasle
        seas=compute_seasonality(df_train['sales'],categorization) # mean value of each categorition   
        series=remove_seasonal_component(df_train["sales"], categorization, seas)
        df_train['sales']=series
        
    df_train=df_train.reset_index(drop=True)
    
    # train
    model=trainer(df_train) # train the time series
    
    # forecast
    forecast=forecaster(model,steps_to_forecast)
    
    # create pandas series based on the forecast resutls
    forecast=pd.Series(forecast)
    forecast.name="sales"
    forecast.index=pd.DatetimeIndex(start='2017-01-01',
                                   freq='D',
                                   periods=forecast.size)
    
    if deseasonize:
        forecast=add_seasonal_component (forecast,categorization,seas)
    final_forecast=pd.DataFrame()
    final_forecast['real_values']=df_validation['sales'][:steps_to_forecast]
    final_forecast['forecast']=forecast
    
    return final_forecast
    
    
    

In [46]:
def extract_single_df(data,store,item):
    """
    extract single store/item time series from provided dataset
    
    :param data: Pandas dataframe with multiple timeseries
    :param store: number of the store
    :param item: number of the item
    :returns: Pandas dataframe with single store/item time series
    
    """
    df_single=data.loc[(data.store==store)&(data.item==item),['date','sales']].copy()  # boolen mask 
    df_single.reset_index(drop=True,inplace=True)
    df_single.date=pd.to_datetime(df_single.date)
    return df_single
     

In [47]:
def compute_all_models(data,ids,categorization,trainer,forecaster, deseasonize, steps_to_forecast=90):
    """
    Train the models and use them to make forecast for all of the individual
    time series separately
    
    :params data: dataframe with multiple time series
    :params ids: list of tuples with stores and items
    :param categorization: Function used to split values into various periods of the season.
    :type categorization: pd.DataFrame -> some categorical type, eg. int
    :param trainer: Function used to train the model
    :type trainer: pd.DataFrame -> model
    :param forecaster: (model, steps) -> prediction
    :param steps_to_forecast: number of steps to forecast
    """
    all_models_forecast={}
    all_models_smape=np.array([])
    number_of_time_series=0
    for store,item in ids:
        single_time_series=extract_single_df(data,store,item)
        predictions=train_and_forecast(single_time_series,categorization,
                                         trainer, forecaster, deseasonize, steps_to_forecast)
        score=smape(predictions['real_values'],predictions['forecast'])
        results={
            "item":item,
            "store":store,
            "predictions":predictions,
            "smape":score
        }
        print (score)
        all_models_smape=np.append(all_models_smape,score)
        all_models_forecast[str(store)+str(item)]=results
        number_of_time_series +=1
    forecast_smape=np.sum(all_models_smape)/number_of_time_series
        
    return all_models_forecast,forecast_smape

    

In [48]:
def smape(y,y_pred):
    """
    compute the SMAPE metrics
    
    :param y: array with true values
    : param y_pred: array with forecasted values
    :returns: average smape metrics for the given periods
    """
    div=(abs(y_pred)+abs(y))/2
    errors=abs(y_pred-y)/div
    
    smape=np.sum(errors)/len(y)
    return smape
    

In [49]:
def compute_avg_smape(df_y,df_y_pred):
    """
    Compute average SMAPE of multiple forecast
    
    :param df_y: data series with real values
    :param df_y_pred: dataframe with multiple forecasts
    :returns: average SMAPE of all forecasts
    """
    avg_smape=0
    for i in range(df_y_pred.shape[1]):
        err=smape(y=df_y.iloc[:,i],
                 y_pred=df_y_pred.iloc[:,i])
        avg_smape +=err
    avg_smape /=df_y_pred.shape[1]
    
    return avg_smape


# Data import and preparation


In [50]:
store_item_data=pd.read_csv("train.csv")
store_item_data.index=pd.DatetimeIndex(store_item_data['date'])
store_item_data.head()

,date,store,item,sales
date,,,,
2013-01-01,2013-01-01,1,1,13
2013-01-02,2013-01-02,1,1,11
2013-01-03,2013-01-03,1,1,14
2013-01-04,2013-01-04,1,1,13
2013-01-05,2013-01-05,1,1,10


In [51]:
stores=store_item_data['store'].unique()
items=store_item_data['item'].unique()
print (stores)
print (items)

[ 1  2  3  4  5  6  7  8  9 10]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50]


In [52]:
number_of_stores=10
number_of_items=50

In [53]:
# create every combination of the store and item.
ids= list(itertools.product(range(1,number_of_stores+1), range(1,number_of_items+1)))
print (ids[:5])
print (len(ids))

[(1, 1), (1, 2), (1, 3), (1, 4), (1, 5)]
500


In [54]:
# an alternative method to create the list ids. 
# temp=np.ones((50,))
# ids=[]
# for i in range(1,len(stores)+1):
#     store_i=i*temp.astype("int32")
#     id_temp=list(zip(store_i,items))
#     ids.extend(id_temp)
# print (ids[:5])

#  Holt-Winters Method

In [55]:
# creat a trainer which can be used to train all the models
# the slice indicate the set for validate. 
hw_trainer=lambda df:smt.ExponentialSmoothing(df.loc[-365:,'sales'],damped=False,seasonal="add",
                                              trend='add',seasonal_periods=7).fit()

In [56]:
# create a forecaster function to get the forcasts for all the models
hw_forecaster=lambda model, steps: model.predict(steps)

In [57]:
hw_results,hw_smape=compute_all_models(store_item_data, ids, categorize_by_week_of_year,
                                         hw_trainer, hw_forecaster, False)
print("Holt Winters method SMAPE on the validation set: ", hw_smape)
# print (hw_results)

c:\python 35\lib\site-packages\statsmodels\tsa\holtwinters.py:725: RuntimeWarning: invalid value encountered in less_equal
  loc = initial_p <= lb
c:\python 35\lib\site-packages\statsmodels\tsa\holtwinters.py:731: RuntimeWarning: invalid value encountered in greater_equal
  loc = initial_p >= ub
c:\python 35\lib\site-packages\ipykernel_launcher.py:47: FutureWarning: Creating a DatetimeIndex by passing range endpoints is deprecated.  Use `pandas.date_range` instead.


0.2859867865835019
0.2082624069078082
0.2125067713662475
0.2793544345604883


c:\python 35\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
c:\python 35\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)


0.26121913570281396
0.21833537417958387
0.21742562865145737
0.21636201080792325
0.19978862117836244
0.1790318605024411
0.19821927156203714
0.19731307919580976
0.16221821218744106
0.21283688487553437
0.17477859852022345
0.24547719461157902
0.2346572962373174
0.20953777885064795
0.19016073246147513
0.20551450010590527
0.23202190275396894
0.21174209675399405
0.2165625747135917
0.17929728738710543
0.16408685729020855
0.20915833133695652
0.2947669273732388
0.20809896369243705
0.18652262112523516
0.24049715205977026
0.21626937846870456
0.24081173744533782
0.21191964464775584
0.24056994284991934
0.17440590462874936
0.17578565842472013
0.2638912562942212
0.18702908114662292
0.21575549021094942
0.27160115599526463


c:\python 35\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)


...

0.24573947103775393
0.1912121882593198
0.2366099157851235
0.22894170059580574
0.22510066053232128
0.1877187308049102
0.26991853263403376
0.19695872025453973
0.17493601316470458
0.23230439725378452
0.3085294587585347
0.19847573550634035
0.18450716834832168
0.23247485070350069
0.21486115071773892
0.22235062461343671
0.20766114682557976
0.25157862476313814


c:\python 35\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)


0.21093883497503094
0.21737966308989545
0.24952476161993112
0.22332099020808102
0.22210087929487116
0.26025000039681606
0.27845484202543946
0.23445807857905454
0.21441803510530844
0.23881668202981254
0.20380143129413836
0.22744255667350619
0.3114558628704438
0.2388955536796235
0.25258061826364236
0.21515876013205812
0.23130873347576503
0.19306729252493954
0.1996719817948748
0.24976118480945672
0.264926337473178
0.18698883824595697
0.18359681492126323
0.17644832596830318
0.17798515191843775
0.1574955074633924
0.19276824215680802
0.18282466861358698
0.18642837854763808
0.19183749435555236
0.17423720133329618
0.2541681419070155
0.21760212131989418
0.17064430731558072
0.2087622236788255
0.1968511470832861
0.19559383359833127
0.18344739988955064
0.1991826746424331
0.20687906864318184
0.17833227537743146
0.17364503747879326
0.23381035248090692


c:\python 35\lib\site-packages\statsmodels\tsa\holtwinters.py:744: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)


0.16738587847444325
0.18410977021416797
0.2157765108676325
0.19277283002393927
0.1879566398875306
0.17372633092200396
0.23023457216201473
0.17960564268686524
0.19296470798214643
0.19582047383050907
0.1749934104481014
0.1995690202607838
0.24099503007046025
0.2476022193047136
0.21078769437625786
0.19116049628794773
0.23140395392380628
0.1587390281808713
0.1870698178733104
0.23116793416550818
0.2094931689554473
0.2011241162286988
0.1955804724087182
0.2351911319166398
0.19689211208202534
0.22615947347475387
0.2808094545934386
0.23666093652929243
0.1862233729600355
0.20935188429113352
0.1752551596640024
0.18572842753936133
0.18083732132101688
0.185018834087
0.18237816096729204
0.17043820318727915
0.20000715630585247
0.1595530700496117
0.2474670979394052
0.21810607790004527
0.1904836376650887
0.1875398032401525
0.2153512178066066
0.2128687719554896
0.19804522344936343
0.25376497659162583
0.19619523290554658
0.17989096236857852
0.19260334780814697
0.2763756092765014
0.18534904816384445
0.1798

In [58]:
print (hw_results["15"])

{'store': 1, 'item': 5, 'smape': 0.26121913570281396, 'predictions':             real_values   forecast
date                              
2017-01-01           25  12.442105
2017-01-02           14  14.129508
2017-01-03           13  11.188786
2017-01-04           15  12.850698
2017-01-05           15  13.379144
2017-01-06           18  13.621398
2017-01-07           15  17.225190
2017-01-08           12  13.429797
2017-01-09           17  14.987972
2017-01-10           11  11.570463
2017-01-11           16  13.853170
2017-01-12            8  13.483681
2017-01-13           10  14.053468
2017-01-14           15  17.052592
2017-01-15           15  14.041023
2017-01-16            7  14.836703
2017-01-17           16  11.809542
2017-01-18           14  14.704315
2017-01-19           17  14.317838
2017-01-20           16  15.088482
2017-01-21           12  17.607000
2017-01-22            9  14.460475
2017-01-23           11  15.137285
2017-01-24           14  12.516440
2017-01-25           